## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Nov. 10th Nov. 17th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,November-13-2024,Condo/Co-op,700 Bayshore Dr #4,Fort Lauderdale,FL,33304.0,300000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/700-...,Beaches MLS,F10467572,N,Y,26.132221,-80.108026


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
108,PAST SALE,2024-11-18,Condo/Co-op,3800 Oaks Clubhouse Dr #411,Pompano Beach,FL,33069.0,192000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/3800-O...,Beaches MLS,F10410543,N,Y,26.221436,-80.175329
55,PAST SALE,2024-11-15,Condo/Co-op,3725 S Ocean Dr #1223,Hollywood,FL,33019.0,349000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/3725-S-Oce...,MARMLS,A11647257,N,Y,25.989969,-80.118043
74,PAST SALE,2024-11-15,Condo/Co-op,5300 Washington St Unit B129,Hollywood,FL,33021.0,85000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/5300-Washi...,MARMLS,A11589533,N,Y,26.000708,-80.196096
28,PAST SALE,2024-11-15,Condo/Co-op,360 Lakeside Ct #215,Fort Lauderdale,FL,33326.0,482000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/360-Lakeside...,Beaches MLS,F10464162,N,Y,26.114518,-80.359266
29,PAST SALE,2024-11-15,Condo/Co-op,2309 Park Ln #105,Hollywood,FL,33021.0,123000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2309-Park-...,Beaches MLS,F10457124,N,Y,26.031891,-80.174030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,PAST SALE,2024-11-12,Condo/Co-op,1007 Republic Ct #1007,Deerfield Beach,FL,33442.0,285000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Deerfield-Beach/1007...,Beaches MLS,F10459041,N,Y,26.297174,-80.157467
54,PAST SALE,2024-11-12,Condo/Co-op,2821 Somerset Dr #407,Lauderdale Lakes,FL,33311.0,64000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Lauderdale-Lakes/282...,MARMLS,A11635331,N,Y,26.160958,-80.190147
70,PAST SALE,2024-11-12,Condo/Co-op,531 N Ocean Blvd #401,Pompano Beach,FL,33062.0,382500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/531-N-...,Beaches MLS,F10451525,N,Y,26.241326,-80.087857
109,PAST SALE,2024-11-12,Condo/Co-op,9141 Sunrise Lakes Blvd #205,Sunrise,FL,33322.0,80000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunrise/9141-Sunrise...,Beaches MLS,F10404850,N,Y,26.163485,-80.275157


In [10]:
df = df.sort_values(by='PRICE',ascending=False)

In [69]:
# df = df.drop(index=64)

In [11]:
df_top_ten = df.head(10)

## Collect Agent Information

In [12]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [13]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [14]:
response_df = pd.DataFrame(response_list)

In [15]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [16]:
df_top_ten = merged_df

## Current Week's Values

In [17]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

109


In [18]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$353,122


In [19]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$282


In [20]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$38,490,251


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [27]:
# Define the date range
# start_date = '2024-09-08'
# end_date = '2024-09-15'

# Filter the DataFrame between two dates (inclusive)
# filtered_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [21]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward
Input Previous Week Condo Sales Total (number units sold):
88
Input Previous Week Condo Average Sales Price:
340709
Input Previous Week Condo Average PSF:
292
Input Previous Week Condo Sales total (ex: 198_000_000)
30_000_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [22]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [23]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [24]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [25]:
pd.set_option('display.max_columns',None)

In [26]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-15,Condo/Co-op,2711 S Ocean Dr #3703,Hollywood,FL,33019.0,2875000.0,3.0,3.5,2711 HOLLYWOOD BEACH COND,3395.0,NaN,2009.0,NaN,847.0,3604.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,MARMLS,A11619462,N,Y,25.996218,-80.117174,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,Harry Polsky,Global Luxury Realty LLC,None,None,Marcelo Violland,Prime International Associates Inc,None,None,1,orange


In [27]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [28]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-15,Condo/Co-op,2711 S Ocean Dr #3703,Hollywood,FL,33019.0,2875000.0,3.0,3.5,2711 HOLLYWOOD BEACH COND,3395.0,NaN,2009.0,NaN,847.0,3604.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,MARMLS,A11619462,N,Y,25.996218,-80.117174,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,Harry Polsky,Global Luxury Realty LLC,None,None,Marcelo Violland,Prime International Associates Inc,None,None,1,orange
1,PAST SALE,2024-11-12,Condo/Co-op,1116 N Ocean Blvd #1602,Pompano Beach,FL,33062.0,2250000.0,2.0,2.5,Solemar,1923.0,NaN,2023.0,NaN,1170.0,2756.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11669074,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,David Marder,"Compass Florida, LLC",None,None,None,None,None,None,2,blue
2,PAST SALE,2024-11-15,Condo/Co-op,1532 SE 12th St #204,Fort Lauderdale,FL,33316.0,1700000.0,3.0,3.5,Vela Vista,2067.0,NaN,2016.0,NaN,822.0,1433.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Beaches MLS,F10437473,N,Y,26.107692,-80.126775,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Sohely Van Woerkom,"Whitaker Realty, PA",Andres Van Woerkom,"Whitaker Realty, PA",Sohely Van Woerkom,"Whitaker Realty, PA",None,None,3,blue
3,PAST SALE,2024-11-13,Condo/Co-op,801 Briny Ave #1204,Pompano Beach,FL,33062.0,1590000.0,3.0,2.5,Sonata Beach Club,2497.0,NaN,2007.0,NaN,637.0,2420.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Beaches MLS,F10457296,N,Y,26.226042,-80.090808,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Stephenie Barbuto,Coldwell Banker Realty,None,None,David M Manupella,"Post and Post, Inc.",Patricia Post,"Post and Post, Inc.",4,blue
4,PAST SALE,2024-11-15,Condo/Co-op,1532 SE 12th St #203,Fort Lauderdale,FL,33316.0,1200000.0,2.0,2.5,Vela Vista,1761.0,NaN,2016.0,NaN,681.0,1259.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Beaches MLS,F10441416,N,Y,26.107692,-80.126775,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Sohely Van Woerkom,"Whitaker Realty, PA",Andres Van Woerkom,"Whitaker Realty, PA",Sohely Van Woerkom,"Whitaker Realty, PA",None,None,5,blue
5,PAST SALE,2024-11-14,Condo/Co-op,77 S Birch Rd Unit 10B,Fort Lauderdale,FL,33316.0,1050000.0,2.0,2.5,Portofino,1435.0,NaN,1982.0,NaN,732.0,1365.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/77-S...,Beaches MLS,F10440799,N,Y,26.122119,-80.106605,https://www.redfin.com/FL/Fort-Lauderdale/77-S...,Brian Justice,Justice & Associates Realty,None,None,Brian Justice,Justice & Associates Realty,None,None,6,blue
6,PAST SALE,2024-11-15,Condo/Co-op,2777 Kinsington Cir Unit 9-4,Weston,FL,33332.0,875000.0,4.0,2.5,LEGENDS AT WESTON HILLS,2340.0,NaN,2002.0,NaN,374.0,766.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/2777-Kinsingt...,MARMLS,A11641421,N,Y,26.093084,-80.385574,https://www.redfin.com/FL/Weston/2777-Kinsingt...,Joanna Jimenez,"Compass Florida, LLC.",None,None,None,None,None,None,7,blue
7,PAST SALE,2024-11-14,Condo/Co-op,411 NW 1st Ave #302,Fort Lauderdale,FL,33301.0,820000.0,2.0,2.0,Foundry Lofts,1512.0,NaN,2007.0,NaN,542.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10458872,N,Y,26.126606,-80.144908,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Timothy Singer,Coldwell Banker Realty,Timothy Powers,Coldwell Banker Realty,None,"Compass Florida, LLC",None,None,8,blue
8,PAST SALE,2024-11-14,Condo/Co-op,2100 S Ocean Ln #1409,Fort Lauderdale,FL,33316.0,732832.0,2.0,2.0,POINT OF AMERICAS CONDO,1513.0,NaN,1969.0,NaN,484.0,1549.0,Sold,

In [30]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [31]:
m.save('index.html')

## Data snagger

In [32]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [33]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-11-15,Condo/Co-op,2711 S Ocean Dr #3703,Hollywood,FL,33019.0,2875000.0,3.0,3.5,2711 HOLLYWOOD BEACH COND,3395.0,NaN,2009.0,NaN,847.0,3604.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,MARMLS,A11619462,N,Y,25.996218,-80.117174,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,Harry Polsky,Global Luxury Realty LLC,None,None,Marcelo Violland,Prime International Associates Inc,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [34]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_11182024


# CREATE TEMPLATE 

In [35]:
muni_set = set(df_top_ten['CITY'])

In [36]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [37]:
df_top_ten.reset_index(inplace=True,drop=True)

In [38]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [39]:
top_sale

'2711 HOLLYWOOD BEACH COND at 2711 S Ocean Dr #3703 in Hollywood'

In [40]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [41]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [42]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-15,Condo/Co-op,2711 S Ocean Dr #3703,Hollywood,FL,33019.0,2875000.0,3.0,3.5,2711 HOLLYWOOD BEACH COND,3395.0,NaN,2009.0,NaN,847.0,3604.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,MARMLS,A11619462,N,Y,25.996218,-80.117174,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,Harry Polsky,Global Luxury Realty LLC,None,None,Marcelo Violland,Prime International Associates Inc,None,None,1,orange,2711 HOLLYWOOD BEACH COND at 2711 S Ocean Dr #...
1,PAST SALE,2024-11-12,Condo/Co-op,1116 N Ocean Blvd #1602,Pompano Beach,FL,33062.0,2250000.0,2.0,2.5,Solemar,1923.0,NaN,2023.0,NaN,1170.0,2756.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11669074,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,David Marder,"Compass Florida, LLC",None,None,None,None,None,None,2,blue,Solemar at 1116 N Ocean Blvd #1602 in Pompano ...
2,PAST SALE,2024-11-15,Condo/Co-op,1532 SE 12th St #204,Fort Lauderdale,FL,33316.0,1700000.0,3.0,3.5,Vela Vista,2067.0,NaN,2016.0,NaN,822.0,1433.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Beaches MLS,F10437473,N,Y,26.107692,-80.126775,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Sohely Van Woerkom,"Whitaker Realty, PA",Andres Van Woerkom,"Whitaker Realty, PA",Sohely Van Woerkom,"Whitaker Realty, PA",None,None,3,blue,Vela Vista at 1532 SE 12th St #204 in Fort Lau...
3,PAST SALE,2024-11-13,Condo/Co-op,801 Briny Ave #1204,Pompano Beach,FL,33062.0,1590000.0,3.0,2.5,Sonata Beach Club,2497.0,NaN,2007.0,NaN,637.0,2420.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Beaches MLS,F10457296,N,Y,26.226042,-80.090808,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Stephenie Barbuto,Coldwell Banker Realty,None,None,David M Manupella,"Post and Post, Inc.",Patricia Post,"Post and Post, Inc.",4,blue,Sonata Beach Club at 801 Briny Ave #1204 in Po...
4,PAST SALE,2024-11-15,Condo/Co-op,1532 SE 12th St #203,Fort Lauderdale,FL,33316.0,1200000.0,2.0,2.5,Vela Vista,1761.0,NaN,2016.0,NaN,681.0,1259.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Beaches MLS,F10441416,N,Y,26.107692,-80.126775,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Sohely Van Woerkom,"Whitaker Realty, PA",Andres Van Woerkom,"Whitaker Realty, PA",Sohely Van Woerkom,"Whitaker Realty, PA",None,None,5,blue,Vela Vista at 1532 SE 12th St #203 in Fort Lau...
5,PAST SALE,2024-11-14,Condo/Co-op,77 S Birch Rd Unit 10B,Fort Lauderdale,FL,33316.0,1050000.0,2.0,2.5,Portofino,1435.0,NaN,1982.0,NaN,732.0,1365.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/77-S...,Beaches MLS,F10440799,N,Y,26.122119,-80.106605,https://www.redfin.com/FL/Fort-Lauderdale/77-S...,Brian Justice,Justice & Associates Realty,None,None,Brian Justice,Justice & Associates Realty,None,None,6,blue,Portofino at 77 S Birch Rd Unit 10B in Fort La...
6,PAST SALE,2024-11-15,Condo/Co-op,2777 Kinsington Cir Unit 9-4,Weston,FL,33332.0,875000.0,4.0,2.5,LEGENDS AT WESTON HILLS,2340.0,NaN,2002.0,NaN,374.0,766.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/2777-Kinsingt...,MARMLS,A11641421,N,Y,26.093084,-80.385574,https://www.redfin.com/FL/Weston/2777-Kinsingt...,Joanna Jimenez,"Compass Florida, LLC.",None,None,None,None,None,None,7,blue,LEGENDS AT WESTON HILLS at 2777 Kinsington Cir...
7,PAST SALE,2024-11-14,Condo/Co-op,411 NW 1st Ave #302,Fort Lauderdale,FL,33301.0,820000.0,2.0,2.0,Foundry Lofts,1512.0,NaN,2007.0,NaN,542.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10458872,N

In [43]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 2711 HOLLYWOOD BEACH COND closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $730,000M to $2,875,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Hollywood, Pompano Beach, Weston

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 109 condo sales totaling $38,490,251 million from Nov. 10th to Nov. 17th. The previous week, brokers closed 88 condo sales totaling $30,000,000.

Last week’s units sold for an average of $353,122, higher than the $340,709 average price from the previous week. The average price per square foot fell 

In [44]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [45]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [46]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [47]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-15,Condo/Co-op,2711 S Ocean Dr #3703,Hollywood,FL,33019.0,2875000.0,3.0,3.5,2711 HOLLYWOOD BEACH COND,3395.0,NaN,2009.0,NaN,847.0,3604.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,MARMLS,A11619462,N,Y,25.996218,-80.117174,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,Harry Polsky,Global Luxury Realty LLC,None,None,Marcelo Violland,Prime International Associates Inc,None,None,1,orange,2711 HOLLYWOOD BEACH COND at 2711 S Ocean Dr #...
1,PAST SALE,2024-11-12,Condo/Co-op,1116 N Ocean Blvd #1602,Pompano Beach,FL,33062.0,2250000.0,2.0,2.5,Solemar,1923.0,NaN,2023.0,NaN,1170.0,2756.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11669074,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,David Marder,"Compass Florida, LLC",None,None,None,None,None,None,2,blue,Solemar at 1116 N Ocean Blvd #1602 in Pompano ...
2,PAST SALE,2024-11-15,Condo/Co-op,1532 SE 12th St #204,Fort Lauderdale,FL,33316.0,1700000.0,3.0,3.5,Vela Vista,2067.0,NaN,2016.0,NaN,822.0,1433.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Beaches MLS,F10437473,N,Y,26.107692,-80.126775,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Sohely Van Woerkom,"Whitaker Realty, PA",Andres Van Woerkom,"Whitaker Realty, PA",Sohely Van Woerkom,"Whitaker Realty, PA",None,None,3,blue,Vela Vista at 1532 SE 12th St #204 in Fort Lau...
3,PAST SALE,2024-11-13,Condo/Co-op,801 Briny Ave #1204,Pompano Beach,FL,33062.0,1590000.0,3.0,2.5,Sonata Beach Club,2497.0,NaN,2007.0,NaN,637.0,2420.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Beaches MLS,F10457296,N,Y,26.226042,-80.090808,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Stephenie Barbuto,Coldwell Banker Realty,None,None,David M Manupella,"Post and Post, Inc.",Patricia Post,"Post and Post, Inc.",4,blue,Sonata Beach Club at 801 Briny Ave #1204 in Po...
4,PAST SALE,2024-11-15,Condo/Co-op,1532 SE 12th St #203,Fort Lauderdale,FL,33316.0,1200000.0,2.0,2.5,Vela Vista,1761.0,NaN,2016.0,NaN,681.0,1259.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Beaches MLS,F10441416,N,Y,26.107692,-80.126775,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Sohely Van Woerkom,"Whitaker Realty, PA",Andres Van Woerkom,"Whitaker Realty, PA",Sohely Van Woerkom,"Whitaker Realty, PA",None,None,5,blue,Vela Vista at 1532 SE 12th St #203 in Fort Lau...
5,PAST SALE,2024-11-14,Condo/Co-op,77 S Birch Rd Unit 10B,Fort Lauderdale,FL,33316.0,1050000.0,2.0,2.5,Portofino,1435.0,NaN,1982.0,NaN,732.0,1365.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/77-S...,Beaches MLS,F10440799,N,Y,26.122119,-80.106605,https://www.redfin.com/FL/Fort-Lauderdale/77-S...,Brian Justice,Justice & Associates Realty,None,None,Brian Justice,Justice & Associates Realty,None,None,6,blue,Portofino at 77 S Birch Rd Unit 10B in Fort La...
6,PAST SALE,2024-11-15,Condo/Co-op,2777 Kinsington Cir Unit 9-4,Weston,FL,33332.0,875000.0,4.0,2.5,LEGENDS AT WESTON HILLS,2340.0,NaN,2002.0,NaN,374.0,766.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/2777-Kinsingt...,MARMLS,A11641421,N,Y,26.093084,-80.385574,https://www.redfin.com/FL/Weston/2777-Kinsingt...,Joanna Jimenez,"Compass Florida, LLC.",None,None,None,None,None,None,7,blue,LEGENDS AT WESTON HILLS at 2777 Kinsington Cir...
7,PAST SALE,2024-11-14,Condo/Co-op,411 NW 1st Ave #302,Fort Lauderdale,FL,33301.0,820000.0,2.0,2.0,Foundry Lofts,1512.0,NaN,2007.0,NaN,542.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10458872,N

In [48]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: 2711 HOLLYWOOD BEACH COND closing tops Broward weekly condo sales 
DEK: Top 10 sales ranged from $730,000M to $2,875,000M
FEATURED HED:
SEO HED: Broward County Weekly Condo Report 
SEO DESCRIPTION: Broward County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #Broward condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward, weekly condo sales, Fort Lauderdale, Hollywood, Pompano Beach, Weston

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County’s NEWS PEG HERE.

Brokers closed 109 condo sales totaling $38,490,251 million from Nov. 10th to Nov. 17th. The previous week, brokers closed 88 condo sales totaling $30,000,000.

Last week’s units sold for an average of $353,122, higher than the $340,709 average price from the previous week. The average price per square foot fell 

In [62]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Hollywood/2711-S-Ocean-Dr-33019/unit-3703/home/42076231


In [63]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Pompano-Beach/1116-N-Ocean-Blvd-33062/unit-1602/home/188241562


In [64]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/2800-E-Sunrise-Blvd-33304/unit-18B/home/41783015


In [65]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Fort-Lauderdale/2100-S-Ocean-Ln-33316/unit-1409/home/41806087


In [66]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Pompano-Beach/1116-N-Ocean-Blvd-33062/unit-1602/home/188241562


In [67]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Weston/2777-Kinsington-Cir-33332/unit-9-4/home/42009024


In [68]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-11-15,Condo/Co-op,2711 S Ocean Dr #3703,Hollywood,FL,33019.0,2875000.0,3.0,3.5,2711 HOLLYWOOD BEACH COND,3395.0,NaN,2009.0,NaN,847.0,3604.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,MARMLS,A11619462,N,Y,25.996218,-80.117174,https://www.redfin.com/FL/Hollywood/2711-S-Oce...,Harry Polsky,Global Luxury Realty LLC,None,None,Marcelo Violland,Prime International Associates Inc,None,None,1,orange,2711 HOLLYWOOD BEACH COND at 2711 S Ocean Dr #...
1,PAST SALE,2024-11-12,Condo/Co-op,1116 N Ocean Blvd #1602,Pompano Beach,FL,33062.0,2250000.0,2.0,2.5,Solemar,1923.0,NaN,2023.0,NaN,1170.0,2756.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,MARMLS,A11669074,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,David Marder,"Compass Florida, LLC",None,None,None,None,None,None,2,blue,Solemar at 1116 N Ocean Blvd #1602 in Pompano ...
2,PAST SALE,2024-11-15,Condo/Co-op,1532 SE 12th St #204,Fort Lauderdale,FL,33316.0,1700000.0,3.0,3.5,Vela Vista,2067.0,NaN,2016.0,NaN,822.0,1433.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Beaches MLS,F10437473,N,Y,26.107692,-80.126775,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Sohely Van Woerkom,"Whitaker Realty, PA",Andres Van Woerkom,"Whitaker Realty, PA",Sohely Van Woerkom,"Whitaker Realty, PA",None,None,3,blue,Vela Vista at 1532 SE 12th St #204 in Fort Lau...
3,PAST SALE,2024-11-13,Condo/Co-op,801 Briny Ave #1204,Pompano Beach,FL,33062.0,1590000.0,3.0,2.5,Sonata Beach Club,2497.0,NaN,2007.0,NaN,637.0,2420.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Beaches MLS,F10457296,N,Y,26.226042,-80.090808,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Stephenie Barbuto,Coldwell Banker Realty,None,None,David M Manupella,"Post and Post, Inc.",Patricia Post,"Post and Post, Inc.",4,blue,Sonata Beach Club at 801 Briny Ave #1204 in Po...
4,PAST SALE,2024-11-15,Condo/Co-op,1532 SE 12th St #203,Fort Lauderdale,FL,33316.0,1200000.0,2.0,2.5,Vela Vista,1761.0,NaN,2016.0,NaN,681.0,1259.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Beaches MLS,F10441416,N,Y,26.107692,-80.126775,https://www.redfin.com/FL/Fort-Lauderdale/1532...,Sohely Van Woerkom,"Whitaker Realty, PA",Andres Van Woerkom,"Whitaker Realty, PA",Sohely Van Woerkom,"Whitaker Realty, PA",None,None,5,blue,Vela Vista at 1532 SE 12th St #203 in Fort Lau...
5,PAST SALE,2024-11-14,Condo/Co-op,77 S Birch Rd Unit 10B,Fort Lauderdale,FL,33316.0,1050000.0,2.0,2.5,Portofino,1435.0,NaN,1982.0,NaN,732.0,1365.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/77-S...,Beaches MLS,F10440799,N,Y,26.122119,-80.106605,https://www.redfin.com/FL/Fort-Lauderdale/77-S...,Brian Justice,Justice & Associates Realty,None,None,Brian Justice,Justice & Associates Realty,None,None,6,blue,Portofino at 77 S Birch Rd Unit 10B in Fort La...
6,PAST SALE,2024-11-15,Condo/Co-op,2777 Kinsington Cir Unit 9-4,Weston,FL,33332.0,875000.0,4.0,2.5,LEGENDS AT WESTON HILLS,2340.0,NaN,2002.0,NaN,374.0,766.0,Sold,NaN,NaN,https://www.redfin.com/FL/Weston/2777-Kinsingt...,MARMLS,A11641421,N,Y,26.093084,-80.385574,https://www.redfin.com/FL/Weston/2777-Kinsingt...,Joanna Jimenez,"Compass Florida, LLC.",None,None,None,None,None,None,7,blue,LEGENDS AT WESTON HILLS at 2777 Kinsington Cir...
7,PAST SALE,2024-11-14,Condo/Co-op,411 NW 1st Ave #302,Fort Lauderdale,FL,33301.0,820000.0,2.0,2.0,Foundry Lofts,1512.0,NaN,2007.0,NaN,542.0,1300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/411-...,Beaches MLS,F10458872,N

## Time on Market Calculator

In [59]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 10, 1) ## List (Earlier) date
date2 = datetime(2024, 11, 12) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

42


## Clean CSV for Datawrapper Chart

In [69]:
chart_df = df_top_ten

In [70]:
chart_df = chart_df.fillna(" ")

In [71]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [72]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [73]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [74]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [75]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [76]:
csv_date_string = today.strftime("%m_%d_%Y")

In [77]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [78]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2024-11-15,2711 S Ocean Dr #3703 in Hollywood,"$2,875,000",3,3.5,"3,395",2009,$847,MARMLS,A11619462,Harry Polsky Global Luxury Realty LLC,Marcelo Violland Prime International Associate...
1,2024-11-12,1116 N Ocean Blvd #1602 in Pompano Beach,"$2,250,000",2,2.5,"1,923",2023,"$1,170",MARMLS,A11669074,"David Marder Compass Florida, LLC",
2,2024-11-15,1532 SE 12th St #204 in Fort Lauderdale,"$1,700,000",3,3.5,"2,067",2016,$822,Beaches MLS,F10437473,"Sohely Van Woerkom Whitaker Realty, PA Andres ...","Sohely Van Woerkom Whitaker Realty, PA"
3,2024-11-13,801 Briny Ave #1204 in Pompano Beach,"$1,590,000",3,2.5,"2,497",2007,$637,Beaches MLS,F10457296,Stephenie Barbuto Coldwell Banker Realty,"David M Manupella Post and Post, Inc. Patricia..."
4,2024-11-15,1532 SE 12th St #203 in Fort Lauderdale,"$1,200,000",2,2.5,"1,761",2016,$681,Beaches MLS,F10441416,"Sohely Van Woerkom Whitaker Realty, PA Andres ...","Sohely Van Woerkom Whitaker Realty, PA"
5,2024-11-14,77 S Birch Rd Unit 10B in Fort Lauderdale,"$1,050,000",2,2.5,"1,435",1982,$732,Beaches MLS,F10440799,Brian Justice Justice & Associates Realty,Brian Justice Justice & Associates Realty
6,2024-11-15,2777 Kinsington Cir Unit 9-4 in Weston,"$875,000",4,2.5,"2,340",2002,$374,MARMLS,A11641421,"Joanna Jimenez Compass Florida, LLC.",
7,2024-11-14,411 NW 1st Ave #302 in Fort Lauderdale,"$820,000",2,2.0,"1,512",2007,$542,Beaches MLS,F10458872,Timothy Singer Coldwell Banker Realty Timothy ...,"Compass Florida, LLC"
8,2024-11-14,2100 S Ocean Ln #1409 in Fort Lauderdale,"$732,832",2,2.0,"1,513",1969,$484,MARMLS,A11627556,Brian Hero One Sotheby's Int'l Realty,Maria Massa London Foster Realty
9,2024-11-13,2800 E Sunrise Blvd Unit 18B in Fort Lauderdale,"$730,000",2,2.0,"1,675",1969,$436,Beaches MLS,F10457093,Thomas Salomone Real Estate II Inc,Terry Behal NextHome Realty Professionals
